In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

from metrics import MAPE_GroupFairnessScorer, evaluate_predictions

c:\Users\Luca\miniconda3\envs\FairTS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Look into autogluon tutorial for explanations of this code  
The dtype of the columns determines the interpretation and preprocessing of the features  
Autogluon has its own dataframe type with static and dynamic features

In [2]:
df = pd.read_csv("possible_datasets/M4/train.csv")
static_features_df = pd.read_csv("possible_datasets/M4/metadata.csv")

WEEKEND_INDICES = [5, 6]
df["weekend"] = pd.DatetimeIndex(df["timestamp"].astype('datetime64[ns]').values).weekday.isin(WEEKEND_INDICES)

train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_features_df,
)

PREDICTION_LENGTH = 14
train_data, test_data = train_data.train_test_split(PREDICTION_LENGTH)      #split seems to be done stratified with respect to the static features

Sorting the dataframe index before generating the train/test split.


# Baseline model

In [4]:
predictor = TimeSeriesPredictor(
    prediction_length=14,
    target="target",    #specify that target is the target and weekend a known active covariate, other dynamic features are automatically detected as known covariates
    known_covariates_names=["weekend"],
    #eval_metric is default WQL
).fit(train_data, presets="fast_training")

Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Luca\Studium\Master\Master-project\AutogluonModels\ag-20251128_104047'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       0.72 GB / 7.88 GB (9.1%)
Disk Space Avail:   81.38 GB / 475.69 GB (17.1%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': 'very_light',
 'known_covariates_names': ['weekend'],
 'num_val_windows': 1,
 'prediction_length': 14,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'D'
Provided train_data has 243060 rows, 100 time series.

In [7]:
df_baseline = evaluate_predictions(test_data, predictor)
df_baseline

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


{'domain':         Industry     Finance       Micro       Other       Macro         std  \
 RMSE -126.281083 -582.310129 -365.533700 -104.385302 -402.593763  179.803758   
 MAE   -76.100385 -267.643768 -173.814771  -58.660795 -248.964606   85.877424   
 MAPE   -0.023005   -0.028483   -0.031536   -0.020588   -0.048381    0.009790   
 
             cv    max_diff   mean_diff  
 RMSE -0.568602  477.924827  246.432467  
 MAE  -0.520353  208.982974  118.166034  
 MAPE -0.322068    0.027793    0.012824  }

# Use Fairness Metric for evalauation

In [8]:
#from metrics import MAPE_GroupFairnessScorer

predictor = TimeSeriesPredictor(
    prediction_length=14,
    target="target",    #specify that target is the target and weekend a known active covariate, other dynamic features are automatically detected as known covariates
    known_covariates_names=["weekend"],
    eval_metric=MAPE_GroupFairnessScorer()# train_data.static_features
).fit(train_data, presets="fast_training")

Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Luca\Studium\Master\Master-project\AutogluonModels\ag-20251128_105219'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       1.09 GB / 7.88 GB (13.9%)
Disk Space Avail:   81.38 GB / 475.69 GB (17.1%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE_GroupFairnessScorer,
 'hyperparameters': 'very_light',
 'known_covariates_names': ['weekend'],
 'num_val_windows': 1,
 'prediction_length': 14,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}



Inferred time series frequency: 'D'
Provided train_data has 243060 rows, 100 time series. Median time series length is 3180 (min=101, max=4301). 

Provided data contains following columns:
	target: 'target'
	known_covariates:
		categorical:        []
		continuous (float): ['weekend']
	static_features:
		categorical:        ['domain']
		continuous (float): []

To learn how to fix incorrectly inferred types, please see documentation for TimeSeriesPredictor.fit

AutoGluon will gauge predictive performance using evaluation metric: 'MAPE_GroupFairnessScorer'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-11-28 11:52:21
Models that will be trained: ['Naive', 'SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'ETS', 'Theta']
Training timeseries model Naive. 
	-0.0244       = Validation score (-MAPE_GroupFairnessScorer)
	0.65    s     = Training runtime
	29.73 

In [9]:
df_regularization = evaluate_predictions(test_data, predictor)
df_regularization["domain"]

Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS
Model not specified in predict, will default to the model with the best validation score: ETS


,Industry,Finance,Micro,Other,Macro,std,cv,max_diff,mean_diff
RMSE,-121.778391,-583.782426,-355.112093,-227.545166,-437.808754,160.845317,-0.465941,462.004035,226.854332
MAE,-73.513268,-267.781804,-168.981017,-100.387109,-274.375031,82.905458,-0.468372,200.861763,113.823644
MAPE,-0.022205,-0.028316,-0.030521,-0.029472,-0.052896,0.010511,-0.321613,0.030691,0.012717


In [10]:
pd.concat([df_baseline["domain"], df_regularization["domain"]], axis=0)

,Industry,Finance,Micro,Other,Macro,std,cv,max_diff,mean_diff
RMSE,-126.281083,-582.310129,-365.533700,-104.385302,-402.593763,179.803758,-0.568602,477.924827,246.432467
MAE,-76.100385,-267.643768,-173.814771,-58.660795,-248.964606,85.877424,-0.520353,208.982974,118.166034
MAPE,-0.023005,-0.028483,-0.031536,-0.020588,-0.048381,0.009790,-0.322068,0.027793,0.012824
RMSE,-121.778391,-583.782426,-355.112093,-227.545166,-437.808754,160.845317,-0.465941,462.004035,226.854332
MAE,-73.513268,-267.781804,-168.981017,-100.387109,-274.375031,82.905458,-0.468372,200.861763,113.823644
MAPE,-0.022205,-0.028316,-0.030521,-0.029472,-0.052896,0.010511,-0.321613,0.030691,0.012717


# Use sampling strategy

In [15]:
# ...existing code...
# Insert after the "Use sampling strategy" markdown cell
import numpy as np
import pandas as pd
import uuid
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import SMOTENC

def oversample_static_and_clone_series(
    ts_df,
    static_df,
    id_col="item_id",
    group_cols=None,
    target_count=None,
    categorical_cols=None,
    random_state=0,
):
    """
    Oversample under-represented groups defined by group_cols using SMOTENC on the static features.
    For each synthetic static sample, clone an existing time series from the same group (nearest/or random)
    and assign a new unique id. Returns augmented (ts_df_aug, static_df_aug).

    - ts_df: original time series dataframe (long format with id_col and timestamp and target)
    - static_df: dataframe keyed by id_col with static features
    - group_cols: list of columns in static_df that define the "group" to balance (e.g. ['Age', 'Gender'])
    - target_count: desired count per group (if None uses max existing group count)
    - categorical_cols: list of categorical columns in static_df (None => infer object dtype)
    """
    if group_cols is None:
        raise ValueError("group_cols must be provided")

    static = static_df.copy().reset_index(drop=True)
    static[id_col] = static[id_col].astype(str)

    # compute group key
    static["_group_key"] = static[group_cols].astype(str).agg("__".join, axis=1)
    counts = static["_group_key"].value_counts().to_dict()
    max_count = max(counts.values())
    if target_count is None:
        target_count = max_count

    # decide which groups to oversample
    to_oversample = {g: target_count - c for g, c in counts.items() if c < target_count}
    if not to_oversample:
        print("No groups need oversampling (already balanced).")
        return ts_df, static_df

    # Prepare features for SMOTENC: use all static columns except the id and temporary group key
    feat_cols = [c for c in static.columns if c not in [id_col, "_group_key"]]
    # infer categorical cols if not given
    if categorical_cols is None:
        categorical_cols = [c for c in feat_cols if static[c].dtype == "object" or str(static[c].dtype).startswith("category")]
    categorical_idx = [feat_cols.index(c) for c in categorical_cols if c in feat_cols]

    # encode categorical cols as ordinal integers (SMOTENC requires integer categories)
    enc = OrdinalEncoder(dtype=int)
    X = static[feat_cols].copy()
    if categorical_cols:
        X[categorical_cols] = enc.fit_transform(X[categorical_cols].astype(str))
    else:
        # ensure numeric array
        X = X.astype(float)

    y = static["_group_key"].values

    # build sampling_strategy dict for classes needing samples
    sampling_strategy = {g: target_count for g in to_oversample.keys()}

    smote = SMOTENC(
        categorical_features=categorical_idx,
        sampling_strategy=sampling_strategy,
        random_state=random_state,
    )

    X_res, y_res = smote.fit_resample(X.values, y)

    # convert to DataFrame for comparison and locating synthetic samples
    X_df = pd.DataFrame(X.values, columns=feat_cols)
    X_res_df = pd.DataFrame(X_res, columns=feat_cols)
    y_res_ser = pd.Series(y_res, name="_group_key")

    # find synthetic rows (those in resampled but not in original)
    # comparing tuples of all features (categorical are integers matching original encoding)
    orig_tuples = set([tuple(row) for row in X_df.itertuples(index=False, name=None)])
    res_tuples = [tuple(row) for row in X_res_df.itertuples(index=False, name=None)]

    synthetic_indices = [i for i, t in enumerate(res_tuples) if t not in orig_tuples]
    if not synthetic_indices:
        print("SMOTENC did not create synthetic samples (unexpected).")
        return ts_df, static_df

    new_static_rows = []
    new_series_rows = []
    # map group key -> list of original ids (to sample donors)
    group_to_ids = static.groupby("_group_key")[id_col].apply(list).to_dict()

    for idx in synthetic_indices:
        row_enc = X_res_df.iloc[idx]
        group_key = y_res_ser.iloc[idx]

        # decode categorical columns back to original labels if necessary
        row_decoded = row_enc.copy()
        if categorical_cols:
            # inverse transform categorical subset
            cat_vals = row_enc[categorical_cols].values.reshape(1, -1)
            cat_decoded = enc.inverse_transform(cat_vals)
            for j, c in enumerate(categorical_cols):
                row_decoded[c] = cat_decoded[0, j]

        # build a new id and static row
        new_id = str(uuid.uuid4())
        new_row = {id_col: new_id}
        for c in feat_cols:
            new_row[c] = row_decoded[c]
        # reconstruct group cols from group_key when possible, otherwise copy from decoded fields
        # (group_key is combination of group_cols separated by "__")
        parts = group_key.split("__")
        for i, gc in enumerate(group_cols):
            if i < len(parts):
                # attempt to cast back to original dtype
                new_row[gc] = parts[i]
        new_static_rows.append(new_row)

        # choose a donor time series from same group (random)
        donors = group_to_ids.get(group_key)
        if not donors:
            # if only synthetic groups (rare) fallback to random original id
            donor_id = static[id_col].iloc[np.random.randint(len(static))]
        else:
            donor_id = np.random.choice(donors)
        # copy all rows in ts_df with donor_id and assign new_id
        donor_rows = ts_df[ts_df[id_col].astype(str) == str(donor_id)].copy()
        if donor_rows.empty:
            # fallback: skip if donor has no series
            continue
        donor_rows[id_col] = new_id
        # optional: jitter the target slightly to avoid exact duplicates (small gaussian noise)
        if "target" in donor_rows.columns:
            donor_rows["target"] = donor_rows["target"].astype(float) * (1.0 + np.random.normal(0, 0.01, size=len(donor_rows)))
        new_series_rows.append(donor_rows)

    if not new_static_rows:
        print("No new static rows created.")
        return ts_df, static_df

    # append new static rows
    static_aug = pd.concat([static_df.reset_index(drop=True), pd.DataFrame(new_static_rows)], ignore_index=True)
    # append new series rows
    ts_aug = pd.concat([ts_df, pd.concat(new_series_rows, ignore_index=True)], ignore_index=True)

    # ensure id types match original
    ts_aug[id_col] = ts_aug[id_col].astype(str)
    static_aug[id_col] = static_aug[id_col].astype(str)

    return ts_aug, static_aug.drop(columns=["_group_key"], errors="ignore")

# Example usage in this notebook
# choose group columns to balance (adjust to your dataset)
GROUP_COLS = ["domain"]  # change as needed
# infer categorical columns automatically; you can pass a list via categorical_cols param
ts_augmented_df, static_augmented_df = oversample_static_and_clone_series(
    df, static_features_df, id_col="item_id", group_cols=GROUP_COLS, target_count=None, random_state=42
)

# rebuild TimeSeriesDataFrame from the augmented datasets
train_data_aug = TimeSeriesDataFrame.from_data_frame(
    ts_augmented_df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=static_augmented_df,
)

print("Original series count:", static_features_df.shape[0])
print("Augmented series count:", static_augmented_df.shape[0])

# ...existing code...

ValueError: SMOTE-NC is not designed to work only with categorical features. It requires some numerical features.